In [1]:
import numpy as np 
import astropy.constants as c
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import gc
import math
#list all the constants used
G=c.G.value
M_sun=c.M_sun.value
M_earth=c.M_earth.value
R_sun=c.R_sun.value
R_earth=c.R_earth.value
hour=3600 #s
day=3600*24
e=0.1
t_LC=1765.5 #s

def delta(Rp, Rs):
    delta = (Rp*R_earth / (Rs*R_sun))**2*1e6
    return delta

def SNR(delta,n,sigma_s):
    SNR=delta*n**0.5/sigma_s
    return SNR

def sigma_s(t, sigma_LC, cdppindex): #t/hour
    t_LC=1765.5/hour 

    return sigma_LC*(t/t_LC)**cdppindex

def t_dur(P,Rs,e,a): #hour
    t_dur = P*day*Rs*R_sun*(1-e**2)/(np.pi*a)/hour
    return t_dur

def a(Ms,P):
    a=((G*Ms*M_sun*(P*day)**2)/(4*np.pi**2))**(1/3)
    return a 

def n(t_obs,P): # the unite of t_obs and P is day
    n=t_obs/P
    return n

def f_n(t_obs,P):
    if t_obs <=2*P:
        return 0
    if t_obs> 2*P and t_obs< 3*P:
        return (t_obs/P-2)
    if t_obs>=3*P:
        return 1
    
def f_eff(SNR):
    if SNR <=6:
        return 0
    if SNR >6 and SNR<=12:
        return (SNR-6)/6
    if SNR>12:
        return 1
    
def f_geo(Rp,Rs,a,e):
    f_geo = (Rp*R_earth+Rs*R_sun)/(a*(1-e**2))
    return f_geo


In [3]:
#1. select M star 

data=pd.read_csv('data/DAI2020STELLAR_MS_NOBINARY.csv',sep=',')

with open('data/DAI2020STELLAR_MS_NOBINARY_M.csv','w') as f:
    s=['KIC',
       'gmag','gmag_err',
       'kmag','kmag_err',
       'parallax','parallax_err',
       'feh','feh_err',
       'RUWE','num_gaia_companions','Kmag_final_prov','evstate',
       
       'iso_mass','iso_mass_err1','iso_mass_err2',
       'iso_teff','iso_teff_err1','iso_teff_err2',
       'iso_logg','iso_logg_err1','iso_logg_err2',
       'iso_feh','iso_feh_err1','iso_feh_err2',
       'iso_rad','iso_rad_err1','iso_rad_err2',
       'iso_rho','iso_rho_err1','iso_rho_err2',
       'iso_lum','iso_lum_err1','iso_lum_err2',
       'iso_age','unReAgeFlag','iso_age_err1','iso_age_err2',
       'iso_dis','iso_dis_err1','iso_dis_err2','iso_avs','iso_gof','TAMS',
       
       'DAI_cdpp3','DAI_cdpp6','DAI_cdpp12',
       'DAI_std_vra','DAI_std_vdec',
       'DAI_mean_vra','DAI_mean_vdec',
       'DAI_sigma_vra','DAI_sigma_vdec',
       'DAI_ra_gaia','DAI_dec_gaia',
       'DAI_parallax','DAI_parallax_error',
       'DAI_pmra','DAI_pmra_error',
       'DAI_pmdec','DAI_pmdec_error',
       
       'dutycycle','dutycycle_post','dataspan','dataspan_post',
       
       'B2018_teff','B2018_teffe',
       'B2018_dis','B2018_disep','B2018_disem',
       'B2018_rad','B2018_radep','B2018_radem',
       'B2018_avs','B2018_evState','B2018_binaryFlag',

      ]
    
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)):
        if data['iso_teff'][i]>=2400 and data['iso_teff'][i]<3700:
            writer.writerow({'KIC':data['KIC'][i],
                             'gmag':data['gmag'][i],
                             'gmag_err':data['gmag_err'][i],
                             'kmag':data['kmag'][i],
                             'kmag_err':data['kmag_err'][i],
                             'parallax':data['parallax'][i],
                             'parallax_err':data['parallax_err'][i],
                             'feh':data['feh'][i],
                             'feh_err':data['feh_err'][i],
                             'RUWE':data['RUWE'][i],
                             'num_gaia_companions':data['num_gaia_companions'][i],
                             'Kmag_final_prov':data['Kmag_final_prov'][i],
                             'evstate':data['evstate'][i],
                             
                             'iso_mass':data['iso_mass'][i],
                             'iso_mass_err1':data['iso_mass_err1'][i],
                             'iso_mass_err2':data['iso_mass_err2'][i],
                             'iso_teff':data['iso_teff'][i],
                             'iso_teff_err1':data['iso_teff_err1'][i],
                             'iso_teff_err2':data['iso_teff_err2'][i],
                             'iso_logg':data['iso_logg'][i],
                             'iso_logg_err1':data['iso_logg_err1'][i],
                             'iso_logg_err2':data['iso_logg_err2'][i],
                             'iso_feh':data['iso_feh'][i],
                             'iso_feh_err1':data['iso_feh_err1'][i],
                             'iso_feh_err2':data['iso_feh_err2'][i],
                             'iso_rad':data['iso_rad'][i],
                             'iso_rad_err1':data['iso_rad_err1'][i],
                             'iso_rad_err2':data['iso_rad_err2'][i],
                             'iso_rho':data['iso_rho'][i],
                             'iso_rho_err1':data['iso_rho_err1'][i],
                             'iso_rho_err2':data['iso_rho_err2'][i],
                             'iso_lum':data['iso_lum'][i],
                             'iso_lum_err1':data['iso_lum_err1'][i],
                             'iso_lum_err2':data['iso_lum_err2'][i],
                             'iso_age':data['iso_age'][i],
                             'unReAgeFlag':data['unReAgeFlag'][i],
                             'iso_age_err1':data['iso_age_err1'][i],
                             'iso_age_err2':data['iso_age_err2'][i],
                             'iso_dis':data['iso_dis'][i],
                             'iso_dis_err1':data['iso_dis_err1'][i],
                             'iso_dis_err2':data['iso_dis_err2'][i],
                             'iso_avs':data['iso_avs'][i],
                             'iso_gof':data['iso_gof'][i],
                             'TAMS':data['TAMS'][i],
                             
                             'DAI_cdpp3':data['DAI_cdpp3'][i],
                             'DAI_cdpp6':data['DAI_cdpp6'][i],
                             'DAI_cdpp12':data['DAI_cdpp12'][i],
                             'DAI_std_vra':data['DAI_std_vra'][i],
                             'DAI_std_vdec':data['DAI_std_vdec'][i],
                             'DAI_mean_vra':data['DAI_mean_vra'][i],
                             'DAI_mean_vdec':data['DAI_mean_vdec'][i],
                             'DAI_sigma_vra':data['DAI_sigma_vra'][i],
                             'DAI_sigma_vdec':data['DAI_sigma_vdec'][i],
                             'DAI_ra_gaia':data['DAI_ra_gaia'][i],
                             'DAI_dec_gaia':data['DAI_dec_gaia'][i],
                             'DAI_parallax':data['DAI_parallax'][i],
                             'DAI_parallax_error':data['DAI_parallax_error'][i],
                             'DAI_pmra':data['DAI_pmra'][i],
                             'DAI_pmra_error':data['DAI_pmra_error'][i],
                             'DAI_pmdec':data['DAI_pmdec'][i],
                             'DAI_pmdec_error':data['DAI_pmdec_error'][i],
                             
                             'dutycycle':data['dutycycle'][i],
                             'dutycycle_post':data['dutycycle_post'][i],
                             'dataspan':data['dataspan'][i],
                             'dataspan_post':data['dataspan_post'][i],
                             
                             'B2018_teff':data['B2018_teff'][i],
                             'B2018_teffe':data['B2018_teffe'][i],
                             'B2018_dis':data['B2018_dis'][i],
                             'B2018_disep':data['B2018_disep'][i],
                             'B2018_disem':data['B2018_disem'][i],
                             'B2018_rad':data['B2018_rad'][i],
                             'B2018_radep':data['B2018_radep'][i],
                             'B2018_radem':data['B2018_radem'][i],
                             'B2018_avs':data['B2018_avs'][i],
                             'B2018_evState':data['B2018_evState'][i],
                             })

/home/dyz/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
data=pd.read_csv('data/DAI2020PLANET_MS_NOBINARY.csv',sep=',')
with open('data/DAI2020PLANET_MS_NOBINARY_M.csv','w') as f:
    s=['KIC',
       'gmag','gmag_err',
       'kmag','kmag_err',
       'parallax','parallax_err',
       'feh','feh_err',
       'RUWE','num_gaia_companions','Kmag_final_prov','evstate',
       
       'iso_mass','iso_mass_err1','iso_mass_err2',
       'iso_teff','iso_teff_err1','iso_teff_err2',
       'iso_logg','iso_logg_err1','iso_logg_err2',
       'iso_feh','iso_feh_err1','iso_feh_err2',
       'iso_rad','iso_rad_err1','iso_rad_err2',
       'iso_rho','iso_rho_err1','iso_rho_err2',
       'iso_lum','iso_lum_err1','iso_lum_err2',
       'iso_age','unReAgeFlag','iso_age_err1','iso_age_err2',
       'iso_dis','iso_dis_err1','iso_dis_err2','iso_avs','iso_gof','TAMS',
       
       'DAI_cdpp3','DAI_cdpp6','DAI_cdpp12',
       'DAI_std_vra','DAI_std_vdec',
       'DAI_mean_vra','DAI_mean_vdec',
       'DAI_sigma_vra','DAI_sigma_vdec',
       'DAI_ra_gaia','DAI_dec_gaia',
       'DAI_parallax','DAI_parallax_error',
       'DAI_pmra','DAI_pmra_error',
       'DAI_pmdec','DAI_pmdec_error',
       
       'dutycycle','dutycycle_post','dataspan','dataspan_post',
       
       'B2018_teff','B2018_teffe',
       'B2018_dis','B2018_disep','B2018_disem',
       'B2018_rad','B2018_radep','B2018_radem',
       'B2018_avs','B2018_evState','B2018_binaryFlag',
       
       'koi_disposition','koi_pdisposition','koi_score',
       'koi_period','koi_period_err1','koi_period_err2',
       'koi_time0bk','koi_time0bk_err1','koi_time0bk_err2',
       'koi_impact','koi_impact_err1','koi_impact_err2',
       'koi_duration','koi_duration_err1','koi_duration_err2',
       'koi_depth','koi_depth_err1','koi_depth_err2',
       'koi_prad','koi_prad_err1','koi_prad_err2',
       'koi_teq','koi_teq_err1','koi_teq_err2',
       'koi_insol','koi_insol_err1','koi_insol_err2',
       'koi_model_snr',
       
       'iso_koi_prad','iso_semi_major_axis','iso_koi_teq',
      ]
    
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(data)): 
        if data['iso_teff'][i]>=2400 and data['iso_teff'][i]<3700:
            writer.writerow({'KIC':data['KIC'][i],
                             'gmag':data['gmag'][i],
                             'gmag_err':data['gmag_err'][i],
                             'kmag':data['kmag'][i],
                             'kmag_err':data['kmag_err'][i],
                             'parallax':data['parallax'][i],
                             'parallax_err':data['parallax_err'][i],
                             'feh':data['feh'][i],
                             'feh_err':data['feh_err'][i],
                             'RUWE':data['RUWE'][i],
                             'num_gaia_companions':data['num_gaia_companions'][i],
                             'Kmag_final_prov':data['Kmag_final_prov'][i],
                             'evstate':data['evstate'][i],
                             
                             'iso_mass':data['iso_mass'][i],
                             'iso_mass_err1':data['iso_mass_err1'][i],
                             'iso_mass_err2':data['iso_mass_err2'][i],
                             'iso_teff':data['iso_teff'][i],
                             'iso_teff_err1':data['iso_teff_err1'][i],
                             'iso_teff_err2':data['iso_teff_err2'][i],
                             'iso_logg':data['iso_logg'][i],
                             'iso_logg_err1':data['iso_logg_err1'][i],
                             'iso_logg_err2':data['iso_logg_err2'][i],
                             'iso_feh':data['iso_feh'][i],
                             'iso_feh_err1':data['iso_feh_err1'][i],
                             'iso_feh_err2':data['iso_feh_err2'][i],
                             'iso_rad':data['iso_rad'][i],
                             'iso_rad_err1':data['iso_rad_err1'][i],
                             'iso_rad_err2':data['iso_rad_err2'][i],
                             'iso_rho':data['iso_rho'][i],
                             'iso_rho_err1':data['iso_rho_err1'][i],
                             'iso_rho_err2':data['iso_rho_err2'][i],
                             'iso_lum':data['iso_lum'][i],
                             'iso_lum_err1':data['iso_lum_err1'][i],
                             'iso_lum_err2':data['iso_lum_err2'][i],
                             'iso_age':data['iso_age'][i],
                             'unReAgeFlag':data['unReAgeFlag'][i],
                             'iso_age_err1':data['iso_age_err1'][i],
                             'iso_age_err2':data['iso_age_err2'][i],
                             'iso_dis':data['iso_dis'][i],
                             'iso_dis_err1':data['iso_dis_err1'][i],
                             'iso_dis_err2':data['iso_dis_err2'][i],
                             'iso_avs':data['iso_avs'][i],
                             'iso_gof':data['iso_gof'][i],
                             'TAMS':data['TAMS'][i],
                                 
                             'DAI_cdpp3':data['DAI_cdpp3'][i],
                             'DAI_cdpp6':data['DAI_cdpp6'][i],
                             'DAI_cdpp12':data['DAI_cdpp12'][i],
                             'DAI_std_vra':data['DAI_std_vra'][i],
                             'DAI_std_vdec':data['DAI_std_vdec'][i],
                             'DAI_mean_vra':data['DAI_mean_vra'][i],
                             'DAI_mean_vdec':data['DAI_mean_vdec'][i],
                             'DAI_sigma_vra':data['DAI_sigma_vra'][i],
                             'DAI_sigma_vdec':data['DAI_sigma_vdec'][i],
                             'DAI_ra_gaia':data['DAI_ra_gaia'][i],
                             'DAI_dec_gaia':data['DAI_dec_gaia'][i],
                             'DAI_parallax':data['DAI_parallax'][i],
                             'DAI_parallax_error':data['DAI_parallax_error'][i],
                             'DAI_pmra':data['DAI_pmra'][i],
                             'DAI_pmra_error':data['DAI_pmra_error'][i],
                             'DAI_pmdec':data['DAI_pmdec'][i],
                             'DAI_pmdec_error':data['DAI_pmdec_error'][i],
                             
                             'dutycycle':data['dutycycle'][i],
                             'dutycycle_post':data['dutycycle_post'][i],
                             'dataspan':data['dataspan'][i],
                             'dataspan_post':data['dataspan_post'][i],
                             
                             'B2018_teff':data['B2018_teff'][i],
                             'B2018_teffe':data['B2018_teffe'][i],
                             'B2018_dis':data['B2018_dis'][i],
                             'B2018_disep':data['B2018_disep'][i],
                             'B2018_disem':data['B2018_disem'][i],
                             'B2018_rad':data['B2018_rad'][i],
                             'B2018_radep':data['B2018_radep'][i],
                             'B2018_radem':data['B2018_radem'][i],
                             'B2018_avs':data['B2018_avs'][i],
                             'B2018_evState':data['B2018_evState'][i],
                             
                             'koi_disposition':data['koi_disposition'][i],
                             'koi_pdisposition':data['koi_pdisposition'][i],
                             'koi_score':data['koi_score'][i],
                             'koi_period':data['koi_period'][i],
                             'koi_period_err1':data['koi_period_err1'][i],
                             'koi_period_err2':data['koi_period_err2'][i],
                             'koi_time0bk':data['koi_time0bk'][i],
                             'koi_time0bk_err1':data['koi_time0bk_err1'][i],
                             'koi_time0bk_err2':data['koi_time0bk_err2'][i],
                             'koi_impact':data['koi_impact'][i],
                             'koi_impact_err1':data['koi_impact_err1'][i],
                             'koi_impact_err2':data['koi_impact_err2'][i],
                             'koi_duration':data['koi_duration'][i],
                             'koi_duration_err1':data['koi_duration_err1'][i],
                             'koi_duration_err2':data['koi_duration_err2'][i],
                             'koi_depth':data['koi_depth'][i],
                             'koi_depth_err1':data['koi_depth_err1'][i],
                             'koi_depth_err2':data['koi_depth_err2'][i],
                             'koi_prad':data['koi_prad'][i],
                             'koi_prad_err1':data['koi_prad_err1'][i],
                             'koi_prad_err2':data['koi_prad_err2'][i],
                             'koi_teq':data['koi_teq'][i],
                             'koi_teq_err1':data['koi_teq_err1'][i],
                             'koi_teq_err2':data['koi_teq_err2'][i],
                             'koi_insol':data['koi_insol'][i],
                             'koi_insol_err1':data['koi_insol_err1'][i],
                             'koi_insol_err2':data['koi_insol_err2'][i],
                             'koi_model_snr':data['koi_model_snr'][i],
                         
                             'iso_koi_prad':data['iso_koi_prad'][i],
                             'iso_semi_major_axis':data['iso_semi_major_axis'][i],
                             'iso_koi_teq':data['iso_koi_teq'][i],
                             })

In [2]:
def stellar_cdpp(): 
    G_stellar_data=pd.read_csv('data/DAI2020STELLAR_MS_NOBINARY_M.csv',sep=',')
    with open('result/cdpp_slop.csv','w+') as f:
        s=['i',
           'KIC',
           'sigma_LC',
           'cdppindex',]
        writer=csv.DictWriter(f,fieldnames=s)
        writer.writeheader()    

        for i in range(0,len(G_stellar_data)):
            x=[3,6,12]
            y=[G_stellar_data['DAI_cdpp3'][i],G_stellar_data['DAI_cdpp6'][i],G_stellar_data['DAI_cdpp12'][i]]
            popt, pcov = curve_fit(sigma_s, x, y)
            sigma_LC=popt[0]
            cdppindex=popt[1]
            writer.writerow({'i':i,
                             'KIC':G_stellar_data['KIC'][i],
                             'sigma_LC':sigma_LC,
                             'cdppindex':cdppindex})
            print(i)

    return 0

stellar_cdpp()

#calcuate the planet occurrence of a given planet

G_planet_data=pd.read_csv('data/DAI2020PLANET_MS_NOBINARY_M.csv',sep=',')
G_stellar_data=pd.read_csv('data/DAI2020STELLAR_MS_NOBINARY_M.csv',sep=',')

######################################################
def N_s(Rp,P,i):
    os.system('mkdir result/DETECTION_EFFICIENCY/')
    N_s_temp=0
    cdpp_data=pd.read_csv('result/cdpp_slop.csv')
    with open('result/DETECTION_EFFICIENCY/'+str(i)+'.csv','w') as f:
        s=['SNR',
           'fn',
           'feff',]
        writer=csv.DictWriter(f,fieldnames=s)
        writer.writeheader()        
        for i in range(0,len(G_stellar_data)):
            a_temp=a(G_stellar_data['iso_mass'][i],P)
            tdur_temp=t_dur(P,G_stellar_data['iso_rad'][i],e,a_temp)
            delta_temp=delta(Rp,G_stellar_data['iso_rad'][i])
            n_temp=n(G_stellar_data['dataspan'][i]*G_stellar_data['dutycycle'][i],P)
            sigma_LC=cdpp_data['sigma_LC'][i]
            cdppindex=cdpp_data['cdppindex'][i]
            sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
            
            SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
            fn_temp=f_n(G_stellar_data['dataspan'][i]*G_stellar_data['dutycycle'][i],P)
            feff_temp=f_eff(SNR_temp)
            writer.writerow({'SNR':SNR_temp,
                             'fn':fn_temp,
                             'feff':feff_temp,})                             
            N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

for i in range(0,len(G_planet_data)):
    if G_planet_data['koi_score'][i]>=0.9 and G_planet_data['iso_koi_prad'][i]>=0:
        N_s(G_planet_data['iso_koi_prad'][i],G_planet_data['koi_period'][i],i)
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [3]:
G_planet_data=pd.read_csv('data/DAI2020PLANET_MS_NOBINARY_M.csv',sep=',')
G_stellar_data=pd.read_csv('data/DAI2020STELLAR_MS_NOBINARY_M.csv',sep=',')


f_occurrence_everyplanet=[]

from subprocess import getoutput
filelist=getoutput('ls result/DETECTION_EFFICIENCY/*.csv')
filename=[]
for line in filelist.split('\n'):
    filename.append(line)
    

    
with open('result/DAI2020_M_OCCURRENCE.csv','w') as f:
    s=['i',
       'kepid',
       'occurrence_rate',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(filename)):
        koi_id=int(filename[i][28:-4])
        P=G_planet_data['koi_period'][koi_id]
        Rp=G_planet_data['iso_koi_prad'][koi_id]
        Ms=G_planet_data['iso_mass'][koi_id]
        Rs=G_planet_data['iso_rad'][koi_id]
        a_temp=a(Ms,P)
        N_inf=pd.read_csv(filename[i],sep=',')
        fn=N_inf['fn']
        feff=N_inf['feff']
        N_s_temp=0
        for j in range(0,len(fn)):
            N_s_temp+=fn[j]*feff[j]
        
        del N_inf,fn,feff
        gc.collect()
        N_s_temp=round(N_s_temp)
        fgeo_temp=f_geo(Rp,Rs,a_temp,e)
        f_occurrence_temp=1./(fgeo_temp*N_s_temp)
        f_occurrence_everyplanet.append(f_occurrence_temp)
        print(koi_id,f_occurrence_temp,G_planet_data['KIC'][koi_id])
        writer.writerow({'i':koi_id,
                         'kepid':G_planet_data['KIC'][koi_id],
                         'occurrence_rate':f_occurrence_temp,})




0 0.020518103221701806 3426367
10 0.033607849512947215 8561063
11 0.022575932115920263 8733898
12 0.05664027225716149 8733898
13 0.042646713398986826 8733898
15 0.03704497174729836 9388479
17 0.049201552767218104 9390653
21 0.030694656894771997 9730163
22 0.025765453736305437 9730163
23 0.4414940947767234 10452252
2 0.06616729015076128 5868793
4 0.14874328088620548 6435936
6 0.015232831364342853 7304449
7 0.028956137607486467 7350067
8 0.024459165105863708 8561063
